In [42]:
# D:\Toppan\jupyter\per machine

import pandas as pd
import os
import numpy as np
from datetime import timedelta
import time
import glob

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

in_dir = 'D:\\Toppan\\2017-11-20 全データ\\処理済(総量)\\vectorized_keikaku_shibata'
files = [f for f in os.listdir(in_dir)]

voc_dir_base = 'D:\\Toppan\\2017-11-20 全データ\\データ\\'


base_out_dir = 'D:\\Toppan\\2017-11-20 全データ\\解析結果(総量)\\年間モデル\\追加学習なし'

# train and test data
energies = ['蒸気', '冷水', '電力']
month = [ '16年11月', '16年12月', '17年1月', '17年2月', '17年3月', '17年4月',
         '17年5月', '17年6月', '17年7月', '17年8月', '17年9月', '17年10月', 
         '17年11月', '17年12月', '18年1月', '18年2月']

month_file = [  '201611010800.xlsx', '201612010800.xlsx', 
                '201701010800.xlsx', '201702010800.xlsx', '201703010800.xlsx',
                '201704010800.xlsx', '201705010800.xlsx', '201706010800.xlsx',
                '201707010800.xlsx', '201708010800.xlsx', '201709010800.xlsx',
                '201710010800.xlsx',  '201711010800.xlsx', '201712010800.xlsx', 
              '201801010800.xlsx', '201802010800.xlsx', ]

In [43]:
def read_df(file, month, sheet_name):
    
    # data
    df = pd.read_excel(os.path.join(in_dir, file), sheet_name=sheet_name)
    
    # April special case
    if month == '17年4月':
        df = df.iloc[:358]
    
    # voc
    voc_dir = os.path.join(voc_dir_base, month, 'VOC再利用追加')
    voc_f = glob.glob(voc_dir + '/*.csv')[0]
    df_voc = pd.read_csv(voc_f, index_col=0, 
                         encoding='shift-jis', 
                         engine='python', 
                         parse_dates=True).fillna(0)
    
    df.columns = [
        '総稼動時間-3', '計画投入数(Ｒ)-3', '外気温度-3', '外気湿度-3', '計画色数-3', '段取-3', 
        '計画停止-3', '計画開始完了数-3', sheet_name + '-3',
        
        '総稼動時間-2', '計画投入数(Ｒ)-2', '外気温度-2', '外気湿度-2', '計画色数-2', '段取-2', 
        '計画停止-2', '計画開始完了数-2', sheet_name + '-2',
        
        '総稼動時間-1', '計画投入数(Ｒ)-1', '外気温度-1', '外気湿度-1', '計画色数-1', '段取-1', 
        '計画停止-1', '計画開始完了数-1', sheet_name + '-1',
        
        'target']
    
    # add voc to data
    df['VOC燃料生成量-3'] = pd.Series(data=df_voc['VOC燃料生成量'].iloc[:-2].values, index=df.index)
    df['VOC燃料生成量-2'] = pd.Series(data=df_voc['VOC燃料生成量'].iloc[1:-1].values, index=df.index)
    df['VOC燃料生成量-1'] = pd.Series(data=df_voc['VOC燃料生成量'].iloc[2:].values, index=df.index)
    
    df['VOC再利用生成量-3'] = pd.Series(data=df_voc['VOC再利用生成量'].iloc[:-2].values, index=df.index)
    df['VOC再利用生成量-2'] = pd.Series(data=df_voc['VOC再利用生成量'].iloc[1:-1].values, index=df.index)
    df['VOC再利用生成量-1'] = pd.Series(data=df_voc['VOC再利用生成量'].iloc[2:].values, index=df.index)
    
    return df

In [44]:
def read_df__(file, month, sheet_name):
    
    # data
    df = pd.read_excel(os.path.join(in_dir, file), sheet_name=sheet_name)
    
    df.columns = [
        '総稼動時間-3', '計画投入数(Ｒ)-3', '外気温度-3', '外気湿度-3', '計画色数-3', '段取-3', 
        '計画停止-3', '計画開始完了数-3', sheet_name + '-3',
        
        '総稼動時間-2', '計画投入数(Ｒ)-2', '外気温度-2', '外気湿度-2', '計画色数-2', '段取-2', 
        '計画停止-2', '計画開始完了数-2', sheet_name + '-2',
        
        '総稼動時間-1', '計画投入数(Ｒ)-1', '外気温度-1', '外気湿度-1', '計画色数-1', '段取-1', 
        '計画停止-1', '計画開始完了数-1', sheet_name + '-1',
        
        'target']
    
    return df

In [45]:
holiday_path = 'D:\\Toppan\\2017-11-20 全データ\\データ\\切り離し全休日\\全休日.xlsx'

def mask_out(X, y, month):
    
    try:
        df_filter = pd.read_excel(holiday_path, sheet_name=month, index_col=0).iloc[2:]
        seisan = True if '生産\n有無' in df_filter else False
        
    except Exception as e:
        print(e, month)
        return X, y
    
    def isBusy(idx):
        row = df_filter.loc[idx]

        if row.loc['切離\n有無'] == '切離' or row.loc['全休\n判定'] == '全休' \
            or row.loc['異常判定'] == '※異常稼動' or row.loc['異常判定'] == '※データ異常' \
            or (seisan and row.loc['生産\n有無'] == '無'):
            return False
        else:
            return True

    x_busy_idx = []
    y_busy_idx = []
    for x_idx, y_idx in zip (X.index, y.index):
        if isBusy(x_idx) and isBusy(y_idx):
            x_busy_idx.append(x_idx)
            y_busy_idx.append(y_idx)

    return X.loc[x_busy_idx], y.loc[y_busy_idx]

In [46]:
def save_fig(g, g_name):

    # seaborn graph
    g.set_xlabel("Relative importance",fontsize=12)
    g.set_ylabel("Features",fontsize=12)
    g.tick_params(labelsize=9)
    g.set_title(g_name + " feature importance")
    g.get_figure().savefig(os.path.join(out_dir, g_name + '.png'))
    

def get_importance_figure(model, name, features):
    
    indices = np.argsort(model.feature_importances_)[::-1]
    
    # save csv
    s = pd.Series(data=model.feature_importances_[indices], 
              index=features[indices])
    s.to_csv(os.path.join(out_dir, name + '_寄与度.csv'), encoding='shift-jis')
    
    return

    # all features
    g = sns.barplot(y=features[indices], 
                    x=model.feature_importances_[indices], 
                    orient='h')
    save_fig(g, name + '_all_features')
    
def get_importance_figure_2(model, name, features):
    
    indices = np.argsort(model.feature_importances_)[::-1]
    
    # features without vapors
    feats, feat_importances = [], []
    for i in indices:
        if name not in features[i]:
            feats.append(features[i])
            feat_importances.append(model.feature_importances_[i])
    
    gg = sns.barplot(y=feats, 
                    x=feat_importances, 
                    orient='h')
    save_fig(g, name + '_without_vapors')

In [47]:
def split_day_night(acc_abs):
    acc_abs_days, acc_abs_nights = [], []
    for i, acc in acc_abs.iteritems():
        if 7 < i.hour < 22:
            acc_abs_days.append(acc)
        else:
            acc_abs_nights.append(acc)

    return acc_abs_days, acc_abs_nights

def get_output(res, output, sname, month):
    res = res[res['target'] != 0]
    
    if len(res) == 0:
        return None
    
    y_pred, y_true = res['preds'], res['target']
    '''calculate abs accuracy'''
    acc_abs = abs(y_pred - y_true) / y_true
    '''aplit days and nights'''
    acc_abs_days, acc_abs_nights = split_day_night(acc_abs)
    len_days, len_nights = len(acc_abs_days), len(acc_abs_nights)

    sname2acc = {'蒸気': [0.2, 0.15], '電力': [0.09, 0.15], '冷水': [0.15, 0.1]}

    '''acc stats'''
    len_acc_days = len(list(filter(lambda x: x <= sname2acc[sname][0], acc_abs_days)))
    len_acc_nights = len(list(filter(lambda x: x <= sname2acc[sname][0], acc_abs_nights)))
    acc_stats_days = len_acc_days / len_days
    acc_stats_nights = len_acc_nights / len_nights

    output['設備名'].append(month + '_' + sname)
    output['平日昼・総'].append(len_days)
    output['平日夜・総'].append(len_nights)
    output['平日昼・基準内'].append(len_acc_days)
    output['平日夜・基準内'].append(len_acc_nights)
    output['平日昼基準率'].append(acc_stats_days)
    output['平日夜基準率'].append(acc_stats_nights)

    return output

In [48]:
def remove_noise(X, Y, energy_name):
    
    if energy_name != '電力':
        return
    
    for idx, row in X.iterrows():
        
        energy_vals = [row[energy_name + '-1'], 
         row[energy_name + '-2'], 
         row[energy_name + '-3']]
        
        if any([x > 10000 for x in energy_vals]):
            
            X.drop([idx], inplace=True)
            Y.drop([idx + timedelta(hours=1)], inplace=True)
    
    for idx, item in Y.iteritems():
        if item > 10000:
            X.drop([idx - timedelta(hours=1)], inplace=True)
            Y.drop([idx], inplace=True)
            
    
    return X, Y

In [49]:
df = read_df__('201711010800.xlsx', '17年11月', '電力')

X_test, Y_test = mask_out(df.drop(columns=['target']).iloc[:-1], 
                          df['target'].iloc[1:], '17年11月')

print(X_test.shape, Y_test.shape)

X_test, Y_test = remove_noise(X_test, Y_test, '電力')

print(X_test.shape, Y_test.shape)



'''
17年5月 2017-05-05 18:00:00 655512
17年5月 2017-05-10 02:00:00 90003955

17年11月 2017-11-28 07:00:00 20006085
'''
#Y_test.index[-100:]

(656, 27) (656,)
(652, 27) (652,)


'\n17年5月 2017-05-05 18:00:00 655512\n17年5月 2017-05-10 02:00:00 90003955\n\n17年11月 2017-11-28 07:00:00 20006085\n'

### 年間データを Leave-One-Out で学習・予測

In [50]:
total_acc = []

energy2name = {'蒸気': 'vapor', '冷水': 'cold water', '電力': 'electric power'}

for m, f in zip(month, month_file):

    #if m  in ['17年7月', '17年8月', '17年9月', '17年10月']: continue
    
    print(m)
    # create output dir
    out_dir = os.path.join(base_out_dir, m)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # set train and test files
    test_month, test_month_file = m, f
    train_month, train_month_file = [x for x in month if x != m], \
                                            [x for x in month_file if x != f]
    
    for energy in energies:
        
        # learning files
        X_learn, Y_learn = [], []
        for f, m in zip(train_month_file, train_month):
            try:
                
                df = read_df__(f, m, energy)
                
            except Exception:
                print(f, m, energy)
            
            x, y = mask_out(df.drop(columns=['target']).iloc[:-1], 
                            df['target'].iloc[1:], m)

            X_learn.append(x)
            Y_learn.append(y)

        X_learn, Y_learn = pd.concat(X_learn), pd.concat(Y_learn)

        # test file
        df = read_df__(test_month_file, test_month, energy)
        X_test, Y_test = mask_out(df.drop(columns=['target']).iloc[:-1], 
                                  df['target'].iloc[1:], test_month)
        
        # remove noise
        if energy == '電力':
            remove_noise(X_learn, Y_learn, '電力')
            remove_noise(X_test, Y_test, '電力')
        
        # model
        model = ExtraTreesRegressor(n_estimators=700, 
                                      n_jobs=-1, 
                                      max_depth=11, 
                                      max_features='auto', 
                                      criterion='mae', 
                                      random_state=700, 
                                      warm_start=True)
        
        # learn 1 hour later target
        start = time.time()
        model.fit(X_learn, Y_learn)
        elapsed_time = time.time() - start

        print(test_month, energy, 'elapsed time: ', elapsed_time, 's')

        # feature importance
        get_importance_figure(model, energy2name[energy], X_test.columns)
        
        # test with online learning
        preds = []
        for idx, row in X_test.iterrows():

            # predict
            preds.append(model.predict(row.values.reshape(1, -1))[0])

            # online learning
            '''
            model.n_estimators += 50

            X_learn = X_learn.append(row)
            Y_learn = Y_learn.append(pd.Series(data=Y_test.loc[idx + timedelta(hours=1)], 
                      index=[idx + timedelta(hours=1)]))

            model.fit(X_learn, Y_learn)
            '''

        # save preds and test
        preds = pd.Series(data=preds, index=Y_test.index, name='preds')
        result = pd.concat([preds, Y_test], axis=1)
        result.to_csv(os.path.join(out_dir, energy + '.csv'), encoding='shift-jis')

        # accuracy
        output = {'設備名': [], 
                  '平日昼・総': [], '平日夜・総': [], 
                  '平日昼・基準内': [], '平日夜・基準内': [], 
                  '平日昼基準率': [], '平日夜基準率': []}
        
        output = get_output(result, output, energy, test_month)
        
        print(test_month, energy, output)

        # save accuracy
        accs = pd.DataFrame(output)
        accs.to_csv(os.path.join(out_dir, energy + '_acc.csv'), index=False, encoding='shift-jis')
        
        total_acc.append(accs)
        
total_acc = pd.concat(total_acc)
total_acc.to_csv(os.path.join(base_out_dir, 'total_acc.csv'), index=False, encoding='shift-jis')

16年11月
16年11月 蒸気 elapsed time:  241.57633662223816 s
16年11月 蒸気 {'設備名': ['16年11月_蒸気'], '平日昼・総': [352], '平日夜・総': [251], '平日昼・基準内': [304], '平日夜・基準内': [233], '平日昼基準率': [0.8636363636363636], '平日夜基準率': [0.9282868525896414]}
16年11月 冷水 elapsed time:  280.9427468776703 s
16年11月 冷水 {'設備名': ['16年11月_冷水'], '平日昼・総': [352], '平日夜・総': [251], '平日昼・基準内': [328], '平日夜・基準内': [216], '平日昼基準率': [0.9318181818181818], '平日夜基準率': [0.8605577689243028]}
16年11月 電力 elapsed time:  363.74807119369507 s
16年11月 電力 {'設備名': ['16年11月_電力'], '平日昼・総': [352], '平日夜・総': [251], '平日昼・基準内': [338], '平日夜・基準内': [246], '平日昼基準率': [0.9602272727272727], '平日夜基準率': [0.9800796812749004]}
16年12月
16年12月 蒸気 elapsed time:  245.44087481498718 s
16年12月 蒸気 {'設備名': ['16年12月_蒸気'], '平日昼・総': [152], '平日夜・総': [243], '平日昼・基準内': [136], '平日夜・基準内': [232], '平日昼基準率': [0.8947368421052632], '平日夜基準率': [0.9547325102880658]}
16年12月 冷水 elapsed time:  288.80213499069214 s
16年12月 冷水 {'設備名': ['16年12月_冷水'], '平日昼・総': [149], '平日夜・総': [243], '平日昼・基準内': [144], '平日夜・基準内': [24

### Error check

In [51]:
for m, f in zip(month, month_file):
    print(m)

    df = read_df(f, '電力')
    x, y = mask_out(df.iloc[:-1, :-1], df.iloc[1:, -1], m)
    
    for i, v in y.iteritems():
        
        if v > 10000:
            print(m, i, v)

16年11月


TypeError: read_df() missing 1 required positional argument: 'sheet_name'

In [ ]:
import os

for subdir, dirs, files in os.walk(base_out_dir):
    for file in files:
        
        if 'without' in file:
            print(os.path.join(subdir, file))
            os.remove(os.path.join(subdir, file))

In [ ]:
res_dir = 'D:\\Toppan\\2017-11-20 全データ\\解析結果(総量)\\年間モデル\\追加学習なし\\ExtraTrees_year'

months = ['17年11月', '17年12月', '18年1月', '18年2月']

holidays = [[3, 23], [23, 29, 30, 31], [1, 2, 3, 4, 8], [10, 11, 12, 25]]

total_acc = []
for m, h in zip(months, holidays):
    for e in energies:
        
        result = pd.read_csv(os.path.join(res_dir, m, e + '.csv'), 
                             index_col=0, parse_dates=True,
                             encoding='shift-jis', engine='python')
        
        result = result.drop([x for x in result.index if x.day in h])
        
        
        # accuracy
        output = {'設備名': [], 
                  '平日昼・総': [], '平日夜・総': [], 
                  '平日昼・基準内': [], '平日夜・基準内': [], 
                  '平日昼基準率': [], '平日夜基準率': []}

        output = get_output(result, output, e, m)

        print(m, e, output)

        # save accuracy
        accs = pd.DataFrame(output)
        total_acc.append(accs)
    
total_acc = pd.concat(total_acc)

total_acc.to_csv(os.path.join(res_dir, '1711-1802_acc.csv'), 
                 index=False, 
                 encoding='shift-jis')
total_acc

In [52]:
744-96-24-20

604

### Collect VOC 2017/11 ~ 2018/2

In [94]:
root_dir = 'D:\\download\\201711_201802'
out_dir = 'D:\\download'

dirs = os.listdir(root_dir)

index = pd.date_range('2017/11/1 8:00', periods=length, freq='H')
df_voc = pd.DataFrame(index=index, columns=['VOC再利用生成量', 'VOC燃料生成量'])

for d in dirs:
    
    file_dir = os.path.join(root_dir, d)
    files = os.listdir(file_dir)
    for f in files:
        #if len(f) != 8: continue
            
        f_path = os.path.join(root_dir, d, f)
        ffs = os.listdir(f_path)
        for ff in ffs:
            
            if 'VOC' not in ff: 
                continue
                
            ff_path = os.path.join(root_dir, d, f, ff)
            
            df = pd.read_csv(ff_path, 
                        encoding='shift-jis', 
                        index_col=0, 
                        parse_dates=True, 
                        engine='python')
            
            if 'VOC再利用生成量' in df:
                df_voc.loc[df.index, 'VOC再利用生成量'] = df['VOC再利用生成量'].iloc[0]
            if 'VOC燃料生成量' in df:
                df_voc.loc[df.index, 'VOC燃料生成量'] = df['VOC燃料生成量'].iloc[0]

df_voc.to_csv(os.path.join(out_dir, 'voc.csv'), 
                          encoding='shift-jis')

print('over')


over


In [128]:
df=pd.read_excel('D:\\download\\VOC全休日雷モード.xlsx', 
                 sheet_name='Sheet1', 
                 encoding='shift-jis', index_col=0)

df.drop(index=['時刻'], inplace=True)
df.to_csv('D:\\download\\VOC全休日雷モード_v2.csv',
                                          encoding='shift-jis')
